# Pakiety do odczytu i zapisu

## openpyxl
### Odczyt z openpyxl

In [1]:
import pandas as pd
import openpyxl
import excel
import datetime as dt

In [2]:
# Otwarcie skoroszytu w celu odczytania wartości komórek.
# Po wczytaniu danych plik jest automatycznie zamykany.
book = openpyxl.load_workbook("xl/stores.xlsx", data_only=True)

In [3]:
# Pobranie obiektu arkusza na podstawie nazwy lub indeksu (liczonego od 0)
sheet = book["2019"]
sheet = book.worksheets[0]

In [4]:
# Pobranie listy z nazwami wszystkich arkuszy
book.sheetnames

['2019', '2020', '2019-2020']

In [5]:
# Pętla przechodząca przez obiekty arkusza.
# Zamiast "name", openpyxl używa "title".
for i in book.worksheets:
    print(i.title)

2019
2020
2019-2020


In [6]:
# Pobranie wymiarów,
# czyli używanego zakresu arkusza
sheet.max_row, sheet.max_column

(8, 6)

In [7]:
# Odczytanie wartości pojedynczej komórki
# przy użyciu notacji "A1" i indeksów komórek (liczonych od 1)
sheet["B6"].value
sheet.cell(row=6, column=2).value

'Boston'

In [8]:
# Wczytanie zakresu wartości komórek przy użyciu naszego modułu excel
data = excel.read(book["2019"], (2, 2), (8, 6))
data[:2]  # Wypisuje dwa pierwsze wiersze

[['Sklep', 'Pracownicy', 'Kierownik', 'Data otwarcia', 'Sklep firmowy'],
 ['New York', 10, 'Sarah', datetime.datetime(2018, 7, 20, 0, 0), False]]

### Zapisywanie z openpyxl

In [9]:
import openpyxl
from openpyxl.drawing.image import Image
from openpyxl.chart import BarChart, Reference
from openpyxl.styles import Font, colors
from openpyxl.styles.borders import Border, Side
from openpyxl.styles.alignment import Alignment
from openpyxl.styles.fills import PatternFill
import excel

In [10]:
# Utworzenie instancji skoroszytu
book = openpyxl.Workbook()

# Pobranie pierwszego arkusza i nadanie mu nazwy
sheet = book.active
sheet.title = "Arkusz1"

# Zapisywanie pojedynczych komórek przy użyciu notacji A1
# i indeksów komórek (liczonych od 1)
sheet["A1"].value = "Witaj 1"
sheet.cell(row=2, column=1, value="Witaj 2")

# Formatowanie: kolor wypełnienia, wyrównanie, obramowanie i czcionka
font_format = Font(color="FF0000", bold=True)
thin = Side(border_style="thin", color="FF0000")
sheet["A3"].value = "Witaj 3"
sheet["A3"].font = font_format
sheet["A3"].border = Border(top=thin, left=thin,
                            right=thin, bottom=thin)
sheet["A3"].alignment = Alignment(horizontal="center")
sheet["A3"].fill = PatternFill(fgColor="FFFF00", fill_type="solid")

# Formatowanie liczb (przy użyciu kodów formatujących Excela)
sheet["A4"].value = 3.3333
sheet["A4"].number_format = "0.00"

# Formatowanie daty (przy użyciu kodów formatujących Excela)
sheet["A5"].value = dt.date(2016, 10, 13)
sheet["A5"].number_format = "yyyy/mm/dd"

# Formuła: należy użyć angielskiej nazwy formuły
# z przecinkami jako separatorami
sheet["A6"].value = "=SUM(A4, 2)"

# Obraz
sheet.add_image(Image("images/python.png"), "C1")

# Lista dwuwymiarowa (używamy naszego modułu excel)
data = [[None, "Północ", "Południe"],
        ["Ubiegły rok", 2, 5],
        ["Bieżący rok", 3, 6]]
excel.write(sheet, data, "A10")

# Wykres
chart = BarChart()
chart.type = "col"
chart.title = "Sprzedaż z podziałem na regiony"
chart.x_axis.title = "Regiony"
chart.y_axis.title = "Sprzedaż"
chart_data = Reference(sheet, min_row=11, min_col=1,
                       max_row=12, max_col=3)
chart_categories = Reference(sheet, min_row=10, min_col=2,
                             max_row=10, max_col=3)
# from_rows interpretuje dane w taki sam sposób,
# jak przy ręcznym dodawaniu wykresu w Excelu
chart.add_data(chart_data, titles_from_data=True, from_rows=True)
chart.set_categories(chart_categories)
sheet.add_chart(chart, "A15")

# Zapisanie skoroszytu tworzy plik na dysku
book.save("openpyxl.xlsx")

In [11]:
book = openpyxl.Workbook()
sheet = book.active
sheet["A1"].value = "To jest szablon"
book.template = True
book.save("template.xltx")

### Edycja z openpyxl

In [12]:
# Wczytanie pliku stores.xlsx, zmiana komórki
# i zapisanie go ze zmianą lokalizacji i (lub) nazwy.
book = openpyxl.load_workbook("xl/stores.xlsx")
book["2019"]["A1"].value = "zmodyfikowano"
book.save("stores_edited.xlsx")

In [13]:
book = openpyxl.load_workbook("xl/macro.xlsm", keep_vba=True)
book["Arkusz1"]["A1"].value = "Kliknij przycisk!"
book.save("macro_openpyxl.xlsm")

## XlsxWriter

In [14]:
import datetime as dt
import xlsxwriter
import excel

In [15]:
# Utworzenie instancji skoroszytu
book = xlsxwriter.Workbook("xlxswriter.xlsx")

# Dodanie arkusza i nadanie mu nazwy
sheet = book.add_worksheet("Arkusz1")

# Zapisywanie pojedynczych komórek przy użyciu notacji A1
# i indeksów komórek (liczonych od 0)
sheet.write("A1", "Witaj 1")
sheet.write(1, 0, "Witaj 2")

# Formatowanie: kolor wypełnienia, wyrównanie, obramowanie i czcionka
formatting = book.add_format({"font_color": "#FF0000",
                              "bg_color": "#FFFF00",
                              "bold": True, "align": "center",
                              "border": 1, "border_color": "#FF0000"})
sheet.write("A3", "Witaj 3", formatting)

# Formatowanie liczb (przy użyciu kodów formatujących Excela)
number_format = book.add_format({"num_format": "0.00"})
sheet.write("A4", 3.3333, number_format)

# Formatowanie daty (przy użyciu kodów formatujących Excela)
date_format = book.add_format({"num_format": "yyyy/mm/dd"})
sheet.write("A5", dt.date(2016, 10, 13), date_format)

# Formuła: należy użyć angielskiej nazwy formuły
# z przecinkami jako separatorami
sheet.write("A6", "=SUM(A4, 2)")

# Obraz
sheet.insert_image(0, 2, "images/python.png")

# Lista dwuwymiarowa (używamy naszego modułu excel)
data = [[None, "Północ", "Południe"],
        ["Ubiegły rok", 2, 5],
        ["Bieżący rok", 3, 6]]
excel.write(sheet, data, "A10")

# Wykres: zajrzyj do pliku "sales_report_xlsxwriter.py"
# z repozytorium towarzyszącego książce, aby zobaczyć
# jak można pracować z użyciem indeksów zamiast adresów komórek
chart = book.add_chart({"type": "column"})
chart.set_title({"name": "Sprzedaż z podziałem na regiony"})
chart.add_series({"name": "=Arkusz1!A11",
                  "categories": "=Arkusz1!B10:C10",
                  "values": "=Arkusz1!B11:C11"})
chart.add_series({"name": "=Arkusz1!A12",
                  "categories": "=Arkusz1!B10:C10",
                  "values": "=Arkusz1!B12:C12"})
chart.set_x_axis({"name": "Regiony"})
chart.set_y_axis({"name": "Sprzedaż"})
sheet.insert_chart("A15", chart)

# Zamknięcie skoroszytu utworzy plik na dysku
book.close()

In [16]:
book = xlsxwriter.Workbook("macro_xlxswriter.xlsm")
sheet = book.add_worksheet("Arkusz1")
sheet.write("A1", "Kliknij przycisk!")
book.add_vba_project("xl/vbaProject.bin")
sheet.insert_button("A3", {"macro": "Witaj", "caption": "Przycisk 1",
                           "width": 130, "height": 35})
book.close()

## pyxlsb

In [17]:
import pyxlsb
import excel

In [18]:
# Pętla po arkuszach. Z pyxlsb, skoroszyt i obiekty arkuszy
# mogą być wykorzystywane jako menedżery kontekstu.
# book.sheets zwraca listę nazw arkuszy, a nie obiektów!
# Aby uzyskać obiekt arkusza, użyj get_sheet().
with pyxlsb.open_workbook("xl/stores.xlsb") as book:
    for sheet_name in book.sheets:
        with book.get_sheet(sheet_name) as sheet:
            dim = sheet.dimension
            print(f"Arkusz '{sheet_name}' ma " 
                  f"{dim.h} wierszy i {dim.w} kolumn")

Arkusz '2019' ma 7 wierszy i 5 kolumn
Arkusz '2020' ma 7 wierszy i 5 kolumn
Arkusz '2019-2020' ma 20 wierszy i 5 kolumn


In [19]:
# Wczytanie wartości z zakresu komórek za pomocą naszego modułu excel.
# Zamiast "2019" można również użyć indeksu arkusza (liczonego od 1).
with pyxlsb.open_workbook("xl/stores.xlsb") as book:
    with book.get_sheet("2019") as sheet:
        data = excel.read(sheet, "B2")
data[:2]  # Wypisuje dwa pierwsze wiersze.

[['Sklep', 'Pracownicy', 'Kierownik', 'Data otwarcia', 'Sklep firmowy'],
 ['New York', 10.0, 'Sarah', 43301.0, False]]

In [20]:
from pyxlsb import convert_date
convert_date(data[1][3])

datetime.datetime(2018, 7, 20, 0, 0)

In [21]:
df = pd.read_excel("xl/stores.xlsb", engine="pyxlsb")

## xlrd, xlwt i xlutils

### Odczyt z xlrd

In [22]:
import xlrd
import xlwt
from xlwt.Utils import cell_to_rowcol2
import xlutils
import excel

In [23]:
# Otwarcie skoroszytu w celu odczytania wartości komórek.
# Po wczytaniu danych plik jest automatycznie zamykany.
book = xlrd.open_workbook("xl/stores.xls")

In [24]:
# Pobranie listy z nazwami wszystkich arkuszy
book.sheet_names()

['2019', '2020', '2019-2020']

In [25]:
# Pętla przechodząca przez obiekty arkusza
for sheet in book.sheets():
    print(sheet.name)

2019
2020
2019-2020


In [26]:
# Pobranie obiektu arkusza na podstawie nazwy lub indeksu (liczonego od 0)
sheet = book.sheet_by_index(0)
sheet = book.sheet_by_name("2019")

In [27]:
# Wymiary
sheet.nrows, sheet.ncols

(8, 6)

In [28]:
# Odczytanie wartości pojedynczej komórki
# przy użyciu notacji "A1" i indeksów komórek (liczonych od 1).
# Znak "*" rozpakowuje krotkę zwracaną przez cell_to_rowcol2
# na pojedyncze argumenty.
sheet.cell(*cell_to_rowcol2("B3")).value
sheet.cell(2, 1).value

'New York'

In [29]:
# Wczytanie zakresu wartości komórek przy użyciu naszego modułu excel
data = excel.read(sheet, "B2")
data[:2]  # Wypisuje dwa pierwsze wiersze

[['Sklep', 'Pracownicy', 'Kierownik', 'Data otwarcia', 'Sklep firmowy'],
 ['New York', 10.0, 'Sarah', datetime.datetime(2018, 7, 20, 0, 0), False]]

### Zapisywanie z xlwt

In [30]:
import xlwt
from xlwt.Utils import cell_to_rowcol2
import datetime as dt
import excel

In [31]:
# Utworzenie instancji skoroszytu
book = xlwt.Workbook()

# Dodanie arkusza i nadanie mu nazwy
sheet = book.add_sheet("Arkusz1")

# Zapisywanie pojedynczych komórek przy użyciu notacji A1
# i indeksów komórek (liczonych od 0)
sheet.write(*cell_to_rowcol2("A1"), "Witaj 1")
sheet.write(r=1, c=0, label="Witaj 2")

# Formatowanie: kolor wypełnienia, wyrównanie, obramowanie i czcionka
formatting = xlwt.easyxf("font: bold on, color red;"
                         "align: horiz center;"
                         "borders: top_color red, bottom_color red,"
                                  "right_color red, left_color red,"
                                  "left thin, right thin,"
                                  "top thin, bottom thin;"
                         "pattern: pattern solid, fore_color yellow;")
sheet.write(r=2, c=0, label="Witaj 3", style=formatting)

# Formatowanie liczb (przy użyciu kodów formatujących Excela)
number_format = xlwt.easyxf(num_format_str="0.00")
sheet.write(3, 0, 3.3333, number_format)

# Formatowanie daty (przy użyciu kodów formatujących Excela)
date_format = xlwt.easyxf(num_format_str="yyyy/mm/dd")
sheet.write(4, 0, dt.datetime(2012, 2, 3), date_format)

# Formuła: należy użyć angielskiej nazwy formuły
# z przecinkami jako separatorami
sheet.write(5, 0, xlwt.Formula("SUM(A4, 2)"))

# Lista dwuwymiarowa (używamy naszego modułu excel)
data = [[None, "Północ", "Południe"],
        ["Ubiegły rok", 2, 5],
        ["Bieżący rok", 3, 6]]
excel.write(sheet, data, "A10")

# Obraz (dopuszczalny jest tylko format bmp)
sheet.insert_bitmap("images/python.bmp", 0, 2)

# Zapisanie pliku na dysk
book.save("xlwt.xls")

### Edytowanie z xlutils

In [32]:
from xlutils.copy import copy

In [33]:
book = xlrd.open_workbook("xl/stores.xls", formatting_info=True)
book = copy(book)
book.get_sheet(0).write(0, 0, "zmienione!")
book.save("stores_edited.xls")

# Zagadnienia zaawansowane
## Praca z dużymi plikami

### Zapis za pomocą openpyxl

In [34]:
book = openpyxl.Workbook(write_only=True)
# Przy write_only=True, book.active nie działa
sheet = book.create_sheet()
# Spowoduje to utworzenie arkusza o wymiarach 1000 x 200 komórek
for row in range(1000):
    sheet.append(list(range(200)))
book.save("openpyxl_optimized.xlsx")

### Zapis za pomocą XlsxWriter

In [35]:
book = xlsxwriter.Workbook("xlsxwriter_optimized.xlsx",
                           options={"constant_memory": True})
sheet = book.add_worksheet()
# Spowoduje to utworzenie arkusza o wymiarach 1000 x 200 komórek
for row in range(1000):
    sheet.write_row(row , 0, list(range(200)))
book.close()

### Odczyt za pomocą xlrd

In [36]:
with xlrd.open_workbook("xl/stores.xls", on_demand=True) as book:
    sheet = book.sheet_by_index(0)  # Wczytuje tylko pierwszy arkusz

In [37]:
with xlrd.open_workbook("xl/stores.xls", on_demand=True) as book:
    with pd.ExcelFile(book, engine="xlrd") as f:
        df = pd.read_excel(f, sheet_name=0)

### Odczyt za pomocą openpyxl

In [38]:
book = openpyxl.load_workbook("xl/big.xlsx",
                              data_only=True, read_only=True,
                              keep_links=False)
# Tutaj należy przeprowadzić żądane operacje odczytu
book.close()  # Wymagane przy read_only=True

### Równoległy odczyt

In [39]:
%%time
data = pd.read_excel("xl/big.xlsx",
                     sheet_name=None, engine="openpyxl")

CPU times: total: 1min 59s
Wall time: 2min


In [40]:
%%time
import parallel_pandas
data = parallel_pandas.read_excel("xl/big.xlsx", sheet_name=None)

CPU times: total: 375 ms
Wall time: 1min 14s


## Formatowanie obiektów DataFrame w Excelu

In [41]:
with pd.ExcelFile("xl/stores.xlsx", engine="openpyxl") as xlfile:
    # Wczytanie obiektu DataFrame
    df = pd.read_excel(xlfile, sheet_name="2020")

    # Pobranie obiektu openpyxl dla skoroszytu
    book = xlfile.book

    # Od tego miejsca mamy kod openpyxl
    sheet = book["2019"]
    value = sheet["B3"].value  # Odczyt pojedynczej wartości

In [42]:
with pd.ExcelWriter("pandas_and_openpyxl.xlsx",
                    engine="openpyxl") as writer:
    df = pd.DataFrame({"kolumna1": [1, 2, 3, 4], "kolumna2": [5, 6, 7, 8]})
    # Zapisanie obiektu DataFrame
    df.to_excel(writer, "Arkusz1", startrow=4, startcol=2)

    # Pobranie obiektów openpyxl dla skoroszytu i arkusza
    book = writer.book
    sheet = writer.sheets["Arkusz1"]

    # Od tego miejsca mamy kod openpyxl
    sheet["A1"].value = "To jest tytuł"  # Zapisanie pojedynczej wartości

In [43]:
# formatowanie indeksu i nagłówków DataFrame
df = pd.DataFrame({"kolumna1": [1, -2], "kolumna2": [-3, 4]},
                   index=["wiersz1", "wiersz2"])
df.index.name = "indeks"
df

,kolumna1,kolumna2
indeks,,
wiersz1,1,-3
wiersz2,-2,4


In [44]:
from openpyxl.styles import PatternFill

In [45]:
with pd.ExcelWriter("formatting_openpyxl.xlsx",
                    engine="openpyxl") as writer:
    # Zapisanie df z domyślnym formatowaniem do A1
    df.to_excel(writer, "Arkusz1", startrow=0, startcol=0)

    # Zapisanie df z niestandardowym formatowaniem indeksu/nagłówka do G2    
    startrow, startcol = 0, 5
    # 1. Zapisanie części z danymi DataFrame
    df.to_excel(writer, "Arkusz1", header=False, index=False,
                startrow=startrow + 1, startcol=startcol + 1)
    # Pobranie obiektu arkusza i utworzenie obiektu stylu
    sheet = writer.sheets["Arkusz1"]
    style = PatternFill(fgColor="D9D9D9", fill_type="solid")

    # 2. Zapisanie stylizowanych nagłówków kolumn
    for i, col in enumerate(df.columns):
        sheet.cell(row=startrow + 1, column=i + startcol + 2,
                   value=col).fill = style

    # 3. Zapisanie stylizowanego indeksu
    index = [df.index.name if df.index.name else None] + list(df.index)
    for i, row in enumerate(index):
        sheet.cell(row=i + startrow + 1, column=startcol + 1,
                   value=row).fill = style

In [46]:
# Formatowanie indeksu/nagłówków za pomocą XlsxWrite
with pd.ExcelWriter("formatting_xlsxwriter.xlsx",
                    engine="xlsxwriter") as writer:
    # Zapisanie df z domyślnym formatowaniem do A1
    df.to_excel(writer, "Arkusz1", startrow=0, startcol=0)

    # Zapisanie df z niestandardowym formatowaniem indeksu/nagłówka do G2
    startrow, startcol = 0, 5
    # 1. Zapisanie części z danymi DataFrame
    df.to_excel(writer, "Arkusz1", header=False, index=False,
                startrow=startrow + 1, startcol=startcol + 1)
    # Pobranie obiektu skoroszytu i arkusza oraz utworzenie obiektu stylu
    book = writer.book
    sheet = writer.sheets["Arkusz1"]
    style = book.add_format({"bg_color": "#D9D9D9"})

    # 2. Zapisanie stylizowanych nagłówków kolumn
    for i, col in enumerate(df.columns):
        sheet.write(startrow, startcol + i + 1, col, style)

    # 3. Zapisanie stylizowanego indeksu
    index = [df.index.name if df.index.name else None] + list(df.index)
    for i, row in enumerate(index):
        sheet.write(startrow + i, startcol, row, style)

In [47]:
# formatowanie części DataFrame zawierającej dane
from openpyxl.styles import Alignment

In [48]:
with pd.ExcelWriter("data_format_openpyxl.xlsx",
                    engine="openpyxl") as writer:
    # Zapisanie DataFrame
    df.to_excel(writer, "Arkusz1")
    
    # Pobranie obiektów skoroszytu i arkusza
    book = writer.book
    sheet = writer.sheets["Arkusz1"]
    
    # Formatowanie pojedynczych komórek
    nrows, ncols = df.shape
    for row in range(nrows):
        for col in range(ncols):
            # +1, aby uwzględnić nagłówek/indeks
            # +1, ponieważ openpyxl indeksuje od 1
            cell = sheet.cell(row=row + 2,
                              column=col + 2)
            cell.number_format = "0.000"
            cell.alignment = Alignment(horizontal="center")

In [49]:
with pd.ExcelWriter("data_format_xlsxwriter.xlsx",
                    engine="xlsxwriter") as writer:
    # Zapisanie DataFrame
    df.to_excel(writer, "Arkusz1")

    # Pobranie obiektów skoroszytu i arkusza    
    book = writer.book
    sheet = writer.sheets["Arkusz1"]
    
    # Formatowanie kolumn (nie można formatować pojedynczych komórek)
    number_format = book.add_format({"num_format": "0.000",
                                     "align": "center"})
    sheet.set_column(first_col=1, last_col=2,
                     cell_format=number_format)

In [50]:
df.style.applymap(lambda x: "number-format: 0.000;"
                            "text-align: center")\
        .to_excel("styled.xlsx")

In [51]:
df = pd.DataFrame({"Date": [dt.date(2020, 1, 1)],
                   "Datetime": [dt.datetime(2020, 1, 1, 10)]})
with pd.ExcelWriter("date.xlsx",
                    date_format="yyyy-mm-dd",
                    datetime_format="yyyy-mm-dd hh:mm:ss") as writer:
    df.to_excel(writer)